In [ ]:
import numpy as np
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
data = pd.read_csv("/content/Alphabets_data.csv")

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
x = data.drop("letter", axis=1)
y = data["letter"]

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
y = pd.DataFrame(y)

In [ ]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
scaled_data = scale.fit_transform(x)

In [ ]:
scaled_data

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(scaled_data, y, test_size =0.2, random_state = 42)

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(x_train.shape[1],)),
    tf.keras.layers.Dense(26, activation='softmax')  # 26 outputs for 26 letters
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))


In [ ]:
_, accuracy = model.evaluate(x_test, y_test)
print('Accuracy: {}'.format(accuracy))


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Get predictions
y_pred_probs = model.predict(x_test)
y_pred = np.argmax(y_pred_probs, axis=1)

# Calculate metrics
precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)


In [ ]:
!pip install tensorflow==2.12.0

In [ ]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

# Function to create the model (needed for KerasClassifier)
def create_model(optimizer='adam', neurons=128):
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(neurons, activation='relu', input_shape=(x_train.shape[1],)),
      tf.keras.layers.Dense(26, activation='softmax')
  ])
  model.compile(optimizer=optimizer,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  return model

# Create the KerasClassifier
model = KerasClassifier(build_fn=create_model, verbose=0)

# Define the hyperparameter grid
param_grid = {
    'batch_size': [16, 32, 64],
    'epochs': [10, 20, 30],
    'optimizer': ['adam', 'rmsprop'],
    'neurons': [64, 128, 256]
}

# Initialize GridSearchCV
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy')

# Perform the grid search
grid_result = grid.fit(x_train, y_train)

# Print the best parameters and score
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# Get the best model
best_model = grid_result.best_estimator_.model

# Evaluate the best model
_, accuracy = best_model.evaluate(x_test, y_test)
print('Accuracy of Best Model: {}'.format(accuracy))

# Predictions and metrics for the best model
y_pred_probs = best_model.predict(x_test)
y_pred = np.argmax(y_pred_probs, axis=1)

precision = precision_score(y_test, y_pred, average='macro')
recall = recall_score(y_test, y_pred, average='macro')
f1 = f1_score(y_test, y_pred, average='macro')

print("Precision (Best Model):", precision)
print("Recall (Best Model):", recall)
print("F1-score (Best Model):", f1)


<ipython-input-17-2f890254f402>:17: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, verbose=0)


167/167 [==============================] - 0s 1ms/step


KeyboardInterrupt: 